## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [47]:
DATA_URL = "https://lib.ru/LITRA/LERMONTOW/geroi.txt"

Устанавливаем библиотеки

In [1]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q tensorflow==2.12
!pip install -q git+https://github.com/dvolchek/rnnmorph.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.3.1 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.23.5 which is incompatible.


Создаём объект морфологического анализатора `RNNMorph`

In [48]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [49]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [50]:
raw_text[:200]

'<html><head><title>Михаил Лермонтов. Герой нашего времени</title></head><body><pre><div align=left><form action=/LITRA/LERMONTOW/geroi.txt><select name=format><OPTION VALUE=".fb2.zip">Fb2.zip<OPTION V'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [51]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [52]:
cleaned_text[:200]

'Михаил Лермонтов. Герой нашего времениFb2.zipEpubСодержаниеFine HTMLPrinted versiontxt(Word,КПК)Lib.ru html\nМихаил Лермонтов. Герой нашего времени\n\n\n     Во всякой книге предисловие есть первая и вмес'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [53]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\graev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 3194 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [54]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if str.isalpha(str(pred.normal_form))] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться

50/50 [==============================] - 18s 352ms/step


sentences: 100%|██████████| 3194/3194 [00:00<00:00, 290353.21it/s]


[['fievre', 'lente', 'медленный', 'горячка', 'франц']]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [55]:
len(predictions)

3194

In [56]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)


41654

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75.

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [41]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\graev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [42]:
fdist = FreqDist(word.lower() for word in non_uniq_tokens)
fdist.most_common(50)[0]

('он', 2211)

In [43]:
counter = 0
for (word, fr) in fdist.most_common(50):
    if word in stopwords.words("russian"):
        counter += 1
counter

35

In [44]:
(50-35)/50

0.3

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [45]:
counter = 0
for (word, fr) in fdist.most_common():
    if fr > 10:
        counter += 1
counter

523

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
